In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats
import random
from matplotlib.patches import Ellipse
import math
import scipy
print(scipy.__version__)

pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_csv('tentamen_data.csv')
#make datetime object, such it can be substrected
df['purchase'] = pd.to_datetime(df['purchase'])
df['redeem'] = pd.to_datetime(df['redeem'])




df['red-pur']=df['redeem'] - df['purchase']
df['t1']='2014-04-07'
df['t1'] = pd.to_datetime(df['t1'])
df['t1-pur']=df['t1']-df['purchase'] 
df_r = df
df_un = df

#remove every not NaT row
df_r = df_r[pd.notnull(df_r['red-pur'])]
#remove every NaT row
df_un = df_un[pd.isnull(df_un['red-pur'])]


#removes days from diff
# df_r['diff'] = df_r['diff'].apply(lambda x: x / np.timedelta64(1,'D'))
df_r['red-pur'] = df_r['red-pur'].astype('timedelta64[D]').astype(int)
df_r['t1-pur'] = df_r['t1-pur'].astype('timedelta64[D]').astype(int)

rp = []
tp= []
for index, row in df_r.iterrows():
    rp.append(row["red-pur"])
    tp.append(row["t1-pur"])
#model for gift card usage
def Pt(p, q, drp):
    return p* np.exp(-q*drp-(p/q)*(1-np.exp(-q*(drp))))

#normalization factor
def Norm(p, q, dtp):
    return 1- np.exp(-(p/q)*(1-np.exp(-q*(dtp))))

#log-likelihood
def LLH(p, q, drp, dtp):
    
    L = - 2 * np.log(Pt(p, q, drp)/Norm(p, q, dtp))
    return L

def calculate_LLH(pq, rp, tp):
    p, q = pq
    LLH_c = 0
    for r,t in zip(rp, tp):
#         print('red-pur: ')
#         print(df_r["red-pur"][i])
#         print('t1-pur: ')
#         print(df_r["t1-pur"][i])
        LLH_c += LLH(p, q, r, t)
#         print('red-pur: ')
#         print(df_r["red-pur"][i])
#         print('t1-pur: ')
#         print(df_r["t1-pur"][i])
#         print(LLH_c)
    return LLH_c
     

def mini():
    min_res = minimize(calculate_LLH, np.array([0.1, 0.1]),args = (rp, tp), method = "SLSQP", bounds = ([0,1],[0,1]))
    [p, q] = min_res.x
    print(min_res)
    return min_res.x
    
# min_res = mini()

# print(df_r.set_index("red-pur"))
# print(calculate_LLH([0.1,0.1], rp, tp))

0.17.1


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('tentamen_data.csv')
df['redeem'] =  pd.to_datetime(df['redeem'])
df['purchase'] =  pd.to_datetime(df['purchase'])
# print df.dtypes
#Df1 the cards with redeem date, ie the cards that have been used
df1 = df[~df.redeem.isnull()]
initialDateTime = pd.to_datetime('20130303', format='%Y%m%d')
# print initialDateTime
df1['purchaseDay'] = (df1.purchase - initialDateTime)
df1['purchaseDay'] = (df1.purchaseDay / np.timedelta64(1, 'D')).astype(int)
df1['redeemDay'] = df1.redeem - initialDateTime
df1['redeemDay'] = (df1.redeemDay / np.timedelta64(1, 'D')).astype(int)
df1['numDays'] = df1.redeem - df1.purchase
df1['numDays'] = (df1.numDays / np.timedelta64(1, 'D')).astype(int)
df2 = df[df.redeem.isnull()]
df2['purchaseDay'] = df2.purchase - initialDateTime
df2['purchaseDay'] = (df2.purchaseDay / np.timedelta64(1, 'D')).astype(int)
# print df1.numDays.describe()
# print 'df1:'
# print df1
# print 'df2:'
# print df2




import scipy.optimize as optimize


endDate = pd.to_datetime('20140407', format='%Y%m%d')
t1 = (endDate - initialDateTime).days
# print t1
def P(tau, p, q):
    return p * np.exp(-q * tau - (p/q) * (1 - np.exp(-q * tau)))
def Norm(t0, t1, p, q):
    return (1 - np.exp(-(p/q) * (1 - np.exp(-q*(t1-t0)))))
def Pr(t0, t, p, q):
    return P(t - t0, p, q)/Norm(t0,t1,p,q)
def LogLikelihood(params, ti0s, tis):
    p = params[0]
    q = params[1]
    PrSum = 0
    for ti0, ti in zip(ti0s, tis):
        PrRow = Pr(ti0, ti, p, q)
        PrSum += np.log(PrRow)
    L = -2 * PrSum
    return L
 
ti0s = [] #purchase days
tis = [] #redeem days
for index, row in df1.iterrows():
    ti0s.append(row.purchaseDay)
    tis.append(row.redeemDay)
    
# print LogLikelihood([0.9, 0.9], ti0s, tis)
# print LogLikelihood([0.5, 0.5], ti0s, tis)
# print LogLikelihood([ 0.19683,  0.13122], ti0s, tis)
# print LogLikelihood([ 0.03525111,  0.02848413], ti0s, tis)
# print LogLikelihood([0.03677187, 0.02829693], ti0s, tis)
# print LogLikelihood([0.03525048, 0.02848451], ti0s, tis)
        
#BFGS, L-BFGS-B, SLSQP
#res = optimize.minimize(LogLikelihood, [0.3,0.3], args=(ti0s, tis))
#print res
p0s = np.arange(0,1, 0.1)
q0s = np.arange(0,1, 0.1)
bestp = -10000
bestq = -10000
bestLogL = 10000000000
for p0 in p0s:
    for q0 in q0s:
        res = optimize.minimize(LogLikelihood, [p0, q0], args=(ti0s, tis), method='SLSQP', bounds=([0,1],[0,1]))
        if(res.fun < bestLogL):
            bestp = res.x[0]
            bestq = res.x[1]
            bestLogL = res.fun
            bestRes = res
            
#res = optimize.minimize(LogLikelihood, [0.10,0.10], args=(ti0s, tis), method ='SLSQP', bounds=([0,1],[0,1]))
#res = optimize.minimize(LogLikelihood, [0.03,0.02], args=(ti0s, tis), method ='SLSQP', bounds=([0,1],[0,1]))
print(res)
# print 'Best p: ' + str(bestp) + ', best q: ' + str(bestq) + ', best loglikelihood: ' + str(bestLogL)
# res = minimize(rosen, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
